In [1]:
#from config import (census_key, gkey)
import numpy as np
import pandas as pd
import requests
import zillow
# import pprint as pp
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from citipy import citipy
import regex as re

In [2]:
#US top 500 school district rankings from --> https://www.alarms.org/best-school-districts-in-america/#

rankings = pd.read_excel('Data/2020alarms.org_top500schooldist.xlsx')  
rankings.head()

,Rank,State,Name
0,1,Illinois,NAPERVILLE CUSD 203
1,2,Illinois,MORTON CUSD 709
2,3,Massachusetts,LEXINGTON
3,4,Arizona,GILBERT UNIFIED DISTRICT
4,5,Massachusetts,WESTFORD


In [3]:
rankings['District'] = [re.sub(r'DISTRICT','', i) for i in rankings['Name']]
rankings['District'] = [re.sub(r'SCHOOLS','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'SCHOOL','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r' CUSD','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r' SD','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r' ISD','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'BOROUGH','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'PUBLIC','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'COMMUNITY','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'COMM','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'BOARD','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'OF','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'EDUCATION','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'THE','', i) for i in rankings['District']]
# rankings['District'] = [re.sub(r',','', i) for i in rankings['District']]
# rankings['District'] = [re.sub(r'-','', i) for i in rankings['District']]
# rankings['District'] = [re.sub(r'.','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'TOWNSHIP','', i) for i in rankings['District']]
rankings.head()

,Rank,State,Name,District
0,1,Illinois,NAPERVILLE CUSD 203,NAPERVILLE 203
1,2,Illinois,MORTON CUSD 709,MORTON 709
2,3,Massachusetts,LEXINGTON,LEXINGTON
3,4,Arizona,GILBERT UNIFIED DISTRICT,GILBERT UNIFIED
4,5,Massachusetts,WESTFORD,WESTFORD


In [4]:
#Repeat name stripping with Regex
#CONSIDER REMOVING ALL SPACES IN DISTRICT NAME - do for both

rankings['District'] = rankings['District'].str.replace(' ', '')
rankings.head()

,Rank,State,Name,District
0,1,Illinois,NAPERVILLE CUSD 203,NAPERVILLE203
1,2,Illinois,MORTON CUSD 709,MORTON709
2,3,Massachusetts,LEXINGTON,LEXINGTON
3,4,Arizona,GILBERT UNIFIED DISTRICT,GILBERTUNIFIED
4,5,Massachusetts,WESTFORD,WESTFORD


In [5]:
rankings[rankings['District'].str.contains("SUMMIT")]

,Rank,State,Name,District
109,110,New Jersey,SUMMIT PUBLIC SCHOOLS,SUMMIT


In [6]:
#US School District data from --> https://catalog.data.gov/dataset/school-district-geographic-relationship-files

districts_zip = pd.read_excel('Data/2015schooldist_byzip.xlsx', encoding='ISO-8859-1',dtype={'ZCTA5CE10': 'str'})  
districts_zip['Zip_code'] = districts_zip['ZCTA5CE10'].apply(lambda x: x.zfill(5))
districts_zip['NAME_LEA15'] = districts_zip['NAME_LEA15'].str.upper() 
districts_zip['strip_name'] = districts_zip['NAME_LEA15'].str.replace(' ', '')

districts_zip.head()

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,strip_name
0,100001,FORT RUCKER SCHOOL DISTRICT,00000,6,52.557308,00000,FORTRUCKERSCHOOLDISTRICT
1,100090,ANNISTON CITY SCHOOL DISTRICT,00000,6,12.844220,00000,ANNISTONCITYSCHOOLDISTRICT
2,100188,CHICKASAW CITY SCHOOL DISTRICT,00000,4,0.000000,00000,CHICKASAWCITYSCHOOLDISTRICT
3,100270,BALDWIN COUNTY SCHOOL DISTRICT,00000,25,135.539762,00000,BALDWINCOUNTYSCHOOLDISTRICT
4,100360,BIBB COUNTY SCHOOL DISTRICT,00000,13,53.863187,00000,BIBBCOUNTYSCHOOLDISTRICT


In [7]:
districts_zip[districts_zip['strip_name'].str.contains("SUMMIT")]

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,strip_name
2126,4900690,NORTH SUMMIT SCHOOL DISTRICT,00000,6,85.185436,00000,NORTHSUMMITSCHOOLDISTRICT
2134,4900990,SOUTH SUMMIT SCHOOL DISTRICT,00000,7,54.326656,00000,SOUTHSUMMITSCHOOLDISTRICT
6535,3415900,SUMMIT SCHOOL DISTRICT,07901,1,5.995824,07901,SUMMITSCHOOLDISTRICT
44298,4670140,SUMMIT SCHOOL DISTRICT 54-6,57251,8,22.132429,57251,SUMMITSCHOOLDISTRICT54-6
44304,4670140,SUMMIT SCHOOL DISTRICT 54-6,57256,8,9.596248,57256,SUMMITSCHOOLDISTRICT54-6
...,...,...,...,...,...,...,...
73067,4900990,SOUTH SUMMIT SCHOOL DISTRICT,84061,7,22.639165,84061,SOUTHSUMMITSCHOOLDISTRICT
73115,4900690,NORTH SUMMIT SCHOOL DISTRICT,84098,6,9.176939,84098,NORTHSUMMITSCHOOLDISTRICT
73117,4900990,SOUTH SUMMIT SCHOOL DISTRICT,84098,7,12.148440,84098,SOUTHSUMMITSCHOOLDISTRICT
83703,5308580,SUMMIT VALLEY SCHOOL DISTRICT,99101,2,54.204974,99101,SUMMITVALLEYSCHOOLDISTRICT


In [8]:


districts_zip['District'] = [re.sub(r'DISTRICT','', i) for i in districts_zip['strip_name']]
districts_zip['District'] = [re.sub(r'SCHOOLS','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'SCHOOL','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'BOROUGH','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'PUBLIC','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'INDEPENDENT','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'COMMUNITY','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'UNIT','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'CUSD','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'OF','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'THE','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'HIGH','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'TOWNSHIP','', i) for i in districts_zip['District']]
# districts_zip['District'] = [re.sub(r'-','', i) for i in districts_zip['District']]
# districts_zip['District'] = [re.sub(r'.','', i) for i in districts_zip['District']]
districts_zip.drop(districts_zip[districts_zip['ZCTA5CE10'] == '00000'].index, inplace = True) 
districts_zip.head()

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,strip_name,District
2343,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00601,132,64.347743,00601,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION
2344,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00602,132,30.612962,00602,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION
2345,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00603,132,31.615793,00603,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION
2346,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00606,132,42.309076,00606,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION
2347,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00610,132,35.915722,00610,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION


In [16]:

merged_df = districts_zip.merge(rankings, on='District', how='left')
# .drop(columns=['strip_name', 'strip_Name'])

merged_df.head()

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,strip_name,District,Rank,State,Name
0,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00601,132,64.347743,00601,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION,NaN,NaN,NaN
1,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00602,132,30.612962,00602,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION,NaN,NaN,NaN
2,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00603,132,31.615793,00603,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION,NaN,NaN,NaN
3,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00606,132,42.309076,00606,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION,NaN,NaN,NaN
4,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00610,132,35.915722,00610,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTEDUCATION,NaN,NaN,NaN


In [17]:
merged_df[merged_df['District'].str.contains("SUMMIT")]

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,strip_name,District,Rank,State,Name
4242,3415900,SUMMIT SCHOOL DISTRICT,07901,1,5.995824,07901,SUMMITSCHOOLDISTRICT,SUMMIT,110.0,New Jersey,SUMMIT PUBLIC SCHOOLS
42067,4670140,SUMMIT SCHOOL DISTRICT 54-6,57251,8,22.132429,57251,SUMMITSCHOOLDISTRICT54-6,SUMMIT54-6,NaN,NaN,NaN
42073,4670140,SUMMIT SCHOOL DISTRICT 54-6,57256,8,9.596248,57256,SUMMITSCHOOLDISTRICT54-6,SUMMIT54-6,NaN,NaN,NaN
42077,4670140,SUMMIT SCHOOL DISTRICT 54-6,57257,8,4.772111,57257,SUMMITSCHOOLDISTRICT54-6,SUMMIT54-6,NaN,NaN,NaN
42091,4670140,SUMMIT SCHOOL DISTRICT 54-6,57263,8,11.714654,57263,SUMMITSCHOOLDISTRICT54-6,SUMMIT54-6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
70861,4900990,SOUTH SUMMIT SCHOOL DISTRICT,84061,7,22.639165,84061,SOUTHSUMMITSCHOOLDISTRICT,SOUTHSUMMIT,NaN,NaN,NaN
70909,4900690,NORTH SUMMIT SCHOOL DISTRICT,84098,6,9.176939,84098,NORTHSUMMITSCHOOLDISTRICT,NORTHSUMMIT,NaN,NaN,NaN
70911,4900990,SOUTH SUMMIT SCHOOL DISTRICT,84098,7,12.148440,84098,SOUTHSUMMITSCHOOLDISTRICT,SOUTHSUMMIT,NaN,NaN,NaN
81497,5308580,SUMMIT VALLEY SCHOOL DISTRICT,99101,2,54.204974,99101,SUMMITVALLEYSCHOOLDISTRICT,SUMMITVALLEY,NaN,NaN,NaN


In [18]:
index = merged_df.index
number_of_rows = len(index)
number_of_rows

82144

In [19]:
clean_merged_df = merged_df[merged_df['Rank'].notna()]
# clean_merged_df["zipcode"] = clean_merged_df["Zip_code"].astype(str).apply(lambda x: x.zfill(5))
# clean_merged_df["zipcode"] = clean_merged_df["Zip_code"].astype(str)
new_merge_df = clean_merged_df.drop(columns=['LEAID', 'ZCTA5CE10','COUNT','LANDAREA','strip_name','Name','District']).rename(columns={"NAME_LEA15": "school_district"}) 
new_merge_df.to_csv('Data/schooldist_rankings_clean.csv')
new_merge_df.head()

,school_district,Zip_code,Rank,State
144,AMHERST-PELHAM SCHOOL DISTRICT,01002,479.0,Massachusetts
148,AMHERST-PELHAM SCHOOL DISTRICT,01003,479.0,Massachusetts
155,TANTASQUA SCHOOL DISTRICT,01010,329.0,Massachusetts
160,SPRINGFIELD SCHOOL DISTRICT,01013,285.0,Pennsylvania
161,SPRINGFIELD SCHOOL DISTRICT,01013,471.0,Pennsylvania


In [13]:
# 

Plan B

In [14]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
#     """
#     :param df_1: the left table to join
#     :param df_2: the right table to join
#     :param key1: key column of the left table
#     :param key2: key column of the right table
#     :param threshold: how close the matches should be to return a match, based on Levenshtein distance
#     :param limit: the amount of matches that will get returned, these are sorted high to low
#     :return: dataframe with boths keys and matches
#     """
#     s = df_2[key2].tolist()

#     m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
#     df_1['matches'] = m

#     m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
#     df_1['matches'] = m2

#     return df_1

In [15]:
# fuzzy_merge(districts_zip, rankings, 'District', 'District', threshold=80)